<div style="text-align: center">
  <img alt="AIcrowd" src="https://gitlab.aicrowd.com/jyotish/pricing-game-notebook-scripts/raw/master/pricing-game-banner.png">
</div>

# How to use this notebook 📝

1. **Copy the notebook**. This is a shared template and any edits you make here will not be saved. _You should copy it into your own drive folder._ For this, click the "File" menu (top-left), then "Save a Copy in Drive". You can edit your copy however you like.
2. **Link it to your AICrowd account**. In order to submit your code to AICrowd, you need to provide your account's API key (see [_"Configure static variables"_](#static-var) for details).
3. **Stick to the function definitions**. The submission to AICrowd will look for the pre-defined function names:
  - `install_packages`
  - `global_imports`
  - `fit_model`
  - `save_model`
  - `load_model`
  - `predict_expected_claim`
  - `predict_premium`
  - `preprocess_X_data`

    Anything else you write outside of these functions will not be part of the final submission (including constants and utility functions), so make sure everything is defined within them, except for:

# Your pricing model 🕵️

In this notebook, you can play with the data, and define and train your pricing model.
You can then directly submit it to the AICrowd server, with some magic code at the end.

### Baseline logistic regression 💪
You can also play with a baseline logistic regression model [implemented here](https://colab.research.google.com/drive/1hbIWxTHri1TP5cffRYzU_z4bQPvKi3FB?usp=sharing). 


# Prepare the notebook 🛠

In [5]:
cat(system('curl -sL https://gitlab.aicrowd.com/jyotish/pricing-game-notebook-scripts/raw/r-functions/r/setup.sh > setup.sh && bash setup.sh', intern=TRUE), sep='\n')
source("aicrowd_helpers.R")

⚙️ Installing AIcrowd utilities...
✅ Installed AIcrowd utilities


# Configure static variables 📎

<a name="static-var"></a>

In order to submit using this notebook, you must visit this URL https://aicrowd.com/participants/me and copy your API key. 

Then you must set the value of `AICROWD_API_KEY` wuth the value.

In [6]:
TRAINING_DATA_PATH = 'training.csv'
AICROWD_API_KEY = 'eaab81e0ad4d64a6b0e7ec99a89205f6'  # You can get the key from https://aicrowd.com/participants/me

# Download dataset files 💾

In [7]:
# Make sure to offically join the challenge and accept the challenge rules! Otherwise you will not be able to download the data
download_aicrowd_dataset(AICROWD_API_KEY) 

💾 Downloading dataset...



# Packages 🗃

Install and require here all the packages you need to define your model. 

**Note**: Installing packages the first time might take some time.

In [8]:
install_packages <- function() {
   install.packages("caret")
   install.packages("rpart")
   install.packages("tidyverse")
   install.packages("xgboost")
   install.packages("mlr")
   install.packages("stringr")
   install.packages("parallel")
   install.packages("parallelMap")
   install.packages("goftest")
   install.packages("tweedie")
}
install_packages()

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘numDeriv’, ‘SQUAREM’, ‘lava’, ‘prodlim’, ‘iterators’, ‘data.table’, ‘gower’, ‘ipred’, ‘timeDate’, ‘foreach’, ‘plyr’, ‘ModelMetrics’, ‘reshape2’, ‘recipes’, ‘pROC’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘fastmatch’, ‘ParamHelpers’, ‘BBmisc’, ‘checkmate’, ‘parallelMap’, ‘XML’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘parallel’ is a base package, and should not be updated”
Installing package into ‘/usr

In [9]:
global_imports <- function() {
   require("caret")
   require("rpart")
   require("tidyverse")
   require("xgboost")
   require("mlr")
   require("stringr")
   require("parallel")
   require("parallelMap")
   require("goftest")
   require("tweedie")
}
global_imports()

Loading required package: caret

Loading required package: lattice

Loading required package: ggplot2

Loading required package: rpart

Loading required package: tidyverse

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0
✔ purrr   0.3.4     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ purrr::lift()   masks caret::lift()

Loading required package: xgboost


Attaching package: ‘xgboost’


The following object is masked from ‘package:dplyr’:

    slice


Loading required package: mlr

Loading required package: ParamHelpers

'mlr' is in maintenance mode since July 2019. Future development
efforts will go into its successor 'mlr3' (<https://mlr3.mlr-org.com>).


Attaching package: ‘mlr’


The following object is masked from ‘package:car

# Loading the data 📲

In [10]:
# Load the dataset.
train_data = read.csv(TRAINING_DATA_PATH)

# Create a model, train it, then save it.
Xdata = within(train_data, rm('claim_amount'))
ydata = train_data['claim_amount']

## How does the data look like? 🔍

In [11]:
glimpse(train_data)

Rows: 228,216
Columns: 26
$ id_policy              <chr> "PL000000", "PL042495", "PL042496", "PL042497"…
$ year                   <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ pol_no_claims_discount <dbl> 0.332, 0.000, 0.196, 0.000, 0.000, 0.000, 0.01…
$ pol_coverage           <chr> "Med2", "Med2", "Med1", "Med2", "Med1", "Med2"…
$ pol_duration           <int> 5, 6, 2, 8, 2, 8, 1, 4, 1, 6, 29, 6, 2, 14, 5,…
$ pol_sit_duration       <int> 1, 1, 1, 5, 2, 2, 1, 2, 1, 3, 1, 3, 2, 1, 2, 3…
$ pol_pay_freq           <chr> "Monthly", "Monthly", "Yearly", "Yearly", "Yea…
$ pol_payd               <chr> "No", "No", "Yes", "No", "No", "No", "No", "No…
$ pol_usage              <chr> "WorkPrivate", "WorkPrivate", "Retired", "Work…
$ drv_sex1               <chr> "M", "M", "M", "F", "F", "F", "M", "F", "M", "…
$ drv_age1               <dbl> 35, 60, 55, 54, 65, 68, 41, 51, 44, 53, 55, 52…
$ drv_age_lic1           <dbl> 16, 41, 35, 31, 38, 46, 20, 7, 22, 34, 34, 32,…
$ drv_drv2               <

In [12]:
as.matrix(head(ydata, 4))

,claim_amount
1,0
2,0
3,0
4,0


# Training the model 🚀

You must first define your first function: `fit_model`. This function takes training data as arguments, and outputs a "model" object -- that you define as you wish. For instance, this could be an array of parameter values. 

You may want to define the function `preprocess_X_data` that prepares and cleans your predictor variables for the training and prediction. 

## Define your data preprocessing

You can add any class or function in this cell for preprocessing. Just make sure that you use the functions here in the `fit_model`, `predict_expected_claim` and `predict_premium` functions if necessary. 


In [13]:
preprocess_X_data <- function (x_raw){
   #' Data preprocessing function: given X_raw, clean the data for training or 
   #' prediction.
   #'
   #' Parameters
   #' ----------
   #' @X_raw : Dataframe, with the columns described in the data dictionary.
   #' 	Each row is a different contract. This data has not been processed.
   #'
   #' Returns
   #' -------
   #' A cleaned / preprocessed version of the dataset
      
   Compress_vh_make_model <- function(x_raw, years=c(1)) {
      #' Keeps only the vehicul models the most relevants
      #' for regression.
      freq_tbl <- x_raw %>%
         select(vh_make_model, year) %>%
         filter(year %in% years) %>%
         table() %>% as.data.frame()
      
      n_labels <- nrow(freq_tbl)
      
      labels_to_concat <- freq_tbl[which(freq_tbl[, 'Freq'] < 30), 1]
      
      x_raw[x_raw[,"vh_make_model"] %in% labels_to_concat, "vh_make_model"] <- 
         'others'

      return(x_raw)
   }
   
   feature_ingeneering <- function(x_raw) {
      x_raw <- Compress_vh_make_model(x_raw)
      x_raw <- x_raw %>% mutate(
         id_policy = NULL,
         across(where(is.character), str_to_lower),
         pop_density = population / town_surface_area,
         vh_speed_drv_age_ratio = vh_speed / drv_age1,
         potential_force_impact = vh_speed * vh_weight,
         pol_pay_freq = NULL
      )
      return(x_raw)
   }
   
   # Add and transform some features
   x_preprocessed <- feature_ingeneering(x_raw)
   
   # Impute missing values
   imputation <- x_preprocessed %>%
      filter(year==1) %>%
      impute(
         classes = list(numeric = imputeMean()),
         dummy.cols = 'vh_value'
      )
   x_preprocessed <- x_preprocessed %>%
         reimpute(imputation$desc)
   
   # One-Hot-Encoding
   x_preprocessed <- model.matrix( ~ . + 0, data = x_preprocessed) %>%
      as.data.frame() %>%
      mutate(vh_value_dummyTRUE = vh_value.dummyTRUE) %>%
      select(-vh_value.dummyTRUE)
    
   return(x_preprocessed)
}

## Define the training logic

In [14]:
fit_model <- function (x_raw, y_raw){
   #' Model training function: given training data (X_raw, y_raw), train this
   #' pricing model.
   #'
   #' Parameters
   #' ----------
   #' @X_raw : Dataframe, with the columns described in the data dictionary.
   #' 	Each row is a different contract. This data has not been processed.
   #' @y_raw : An array, with the value of the claims, in the same order as 
   #'    contracts in X_raw. A one dimensional array, with values either 0 
   #'    (most entries) or >0.
   #'
   #' Returns
   #' -------
   #' self: (optional), this instance of the fitted model.
   
   TRAINING_YEARS <-  c(1,2,3,4)
   VALIDATION_YEARS <- c(4)
   GRIDSEARCH <- FALSE
   EVALUATE_MODEL <- FALSE
   
   x_raw <- Xdata
   y_raw <- ydata
   
   x_clean  <- preprocess_X_data(x_raw)
   df <- data.frame(y_raw, x_clean)
   
   df_valid <- df %>% filter(year %in% VALIDATION_YEARS) %>% select(-c(year))
   df_train <- df %>% filter(year %in% TRAINING_YEARS) %>% select(-c(year))
   
   # Training an occurrence detection model with XGB
   train_xgb_occurrence <- 
      function(df_train, df_valid, gridsearch=F, evaluate_perf=T){
         #' Function to train a XGBoost with an objective of type 
         #' binary:logistic for occurrence detection.
         #' 
         #' Parameters
         #' ----------
         #' @df_train dataframe containing training data
         #' @df_valid dataframe containing valitation data
         #' @gridsearch Boolean: Use Gridsearch to optimize hyperparameters ?
         #' @evaluate_perf Boolean: Use df_valid to calculate ROC AUC on 
         #' predictions
         #' 
         #' returns
         #' -------
         #' @model trained model
         #' @auc Calculated ROC AUC on the validation data.
         
         # Rebalancing the classes for the training set
         resample <- function(df, strategy=list('under'=0.7, 'over'=1.2)) {
            
            random_undersample <- function(df, size){
               df0 <- df %>% filter(occ == F) %>%
                  slice_sample(n=size, replace=FALSE)
               df1 <- df %>% filter(occ == T)
               return(rbind(df0, df1))
            }
            
            random_oversample <- function(df, size){
               df0 <- df %>% filter(occ == F) 
               df1 <- df %>% filter(occ == T) %>%
                  slice_sample(n=size, replace=TRUE)
               return(rbind(df0, df1))
            }
               
            dim_classes <- df %>% select(occ) %>% table()
            dim_classes <- dim_classes * c(strategy$under, strategy$over)
            
            df <- random_oversample(df, dim_classes[2])
            df <- random_undersample(df, dim_classes[1])
            return(df)
         }
         
         
         df_train_occ <- df_train %>%
            mutate(occ = factor(claim_amount>0)) %>%
            select(-claim_amount) %>% 
            resample()
            
         # Preparing a validation dataset
         df_valid_occ <- df_valid %>%
            mutate(occ = factor(claim_amount>0)) %>%
            select(-claim_amount)
      
         # Defining task and learner for the mlr optimizer
         train_task <- makeClassifTask(
            data = df_train_occ, target = "occ", positive = "FALSE")
         valid_task <- makeClassifTask(
            data = df_valid_occ, target = "occ", positive = "FALSE")
         
         learner <- makeLearner("classif.xgboost", predict.type = "prob")
         
         if (gridsearch) {
            # Set parallel backend
            parallelStartSocket(cpus = detectCores())
            
            # Model fixed parameters
            learner$par.vals <- list(
               objective = "binary:logistic",
               eval_metric = "auc",
               nrounds = 100
            )
            
            # Set parameter space for gridsearch
            params <- makeParamSet(
               makeDiscreteParam("booster", values = c(
                  'gbtree', 'gblinear', 'dart')),
               makeIntegerParam("max_depth", lower = 3L, upper = 6L),
               makeNumericParam("min_child_weight", lower = 1L, upper = 10L),
               makeNumericParam("subsample", lower = 0.5, upper = 1),
               makeNumericParam("colsample_bytree", lower = 0.5, upper = 1),
               makeNumericParam("lambda", lower=1, upper=4),
               makeNumericParam("alpha", lower=0, upper=3),
               makeNumericParam("eta", lower = 0.1, upper = 0.5)
            )
            
            resampling <- makeResampleDesc("CV", stratify = T, iters = 5L)
            control <- makeTuneControlRandom(maxit = 10L, tune.threshold=T)
            
            # Parameter tuning
            mytune <- tuneParams(
               learner = learner,
               task = train_task,
               resampling = resampling,
               par.set = params,
               control = control,
               show.info = T
            )
            parallelStop()
            
            # Set hyperparameters
            learner <- setHyperPars(learner, par.vals = mytune$x)
            
         } else {
            # Pretuned parameters :
            # ---------------------
            #' [Tune] Result: booster=gbtree; max_depth=6; 
            #' min_child_weight=2.79; subsample=0.749; colsample_bytree=0.584; 
            #' lambda=3.93; alpha=0.0535; eta=0.469 : 
            #' @mmce.test.mean=0.1655115
            learner$par.vals <- list(
               objective = "binary:logistic",
               eval_metric = "auc",
               nrounds = 50,
               eta = 0.1,
               booster = 'gbtree',
               max_depth = 3,
               min_child_weight = 2.79,
               subsample = 0.749,
               colsample_bytree = 0.584,
               lambda = 3.93,
               alpha = 2
            )
         }
         
         # Train model
         xgb_model <- mlr::train(learner = learner, task = train_task)
         
         # Evaluation of performance for the occurrence detection.
         if (evaluate_perf){
            # Predictions
            occ_predictions <- predict(xgb_model, valid_task)
            # AUC
            occ_auc <- mlr::performance(occ_predictions, mlr::auc)
            print(occ_auc, quote=F)
            # Confusion matrix
            print(mlr::calculateConfusionMatrix(pred = occ_predictions), quote=F)
            
            return(list("model"=xgb_model, "auc"=occ_auc))
            }
         return(list("model"=xgb_model))
   }
   
   # Training aggregated severity model (XGB: Tweedie model)
   train_xgb_tweedie <- 
      function(df_train, df_valid, gridsearch=F, evaluate_perf=T){
         #' Function to train a XGBoost with an objective of type 
         #' reg:tweedie
         #' 
         #' Parameters
         #' ----------
         #' @df_train dataframe containing training data
         #' @df_valid dataframe containing valitation data
         #' @gridsearch Boolean: Use Gridsearch to optimize hyperparameters ?
         #' @evaluate_perf Boolean: Use df_valid to calculate RMSE on 
         #' predictions
         #' 
         #' returns
         #' -------
         #' @model trained model
         #' @rmse Calculated rmse on the validation data.
         
         df_train_loss <- df_train %>%
            filter(claim_amount>0)
         
         df_valid_loss <- df_valid %>%
            filter(claim_amount>0)
         
         # Defining task and learner for the mlr optimizer
         train_task <- makeRegrTask(
            data = df_train_loss, 
            target = "claim_amount")
         valid_task <- makeRegrTask(
            data = df_valid_loss, 
            target = "claim_amount")
         
         learner <- makeLearner("regr.xgboost", predict.type = "response")
         
         if (gridsearch) {
            # Set parallel backend
            parallelStartSocket(cpus = detectCores())
            
            # Model fixed parameters
            learner$par.vals <- list(
               objective = "reg:tweedie",
               eval_metric = "rmse",
               nrounds = 200,
               gamma = 1e-1
            )
            
            # Set parameter space for gridsearch
            params <- makeParamSet(
               makeNumericParam("tweedie_variance_power", lower=1, upper=2),
               makeDiscreteParam("booster", values = c(
                  'gbtree', 'gblinear', 'dart')),
               makeIntegerParam("max_depth", lower = 3L, upper = 6L),
               makeNumericParam("min_child_weight", lower = 1L, upper = 10L),
               makeNumericParam("subsample", lower = 0.5, upper = 1),
               makeNumericParam("colsample_bytree", lower = 0.5, upper = 1),
               makeNumericParam("lambda", lower=1, upper=4),
               makeNumericParam("alpha", lower=0, upper=3),
               makeNumericParam('eta', lower=0.1, upper=0.5)
            )
            
            control <- makeTuneControlRandom(maxit = 10L)
            resampling <- makeResampleDesc("CV", iters = 5L)
            
            # Parameter tuning
            mytune <- tuneParams(
               learner = learner,
               task = train_task,
               resampling = resampling,
               par.set = params,
               control = control,
               show.info = T
            )
            parallelStop()
            
            # Set hyperparameters
            learner <- setHyperPars(learner, par.vals = mytune$x)
            
         } else {
            # Pretuned parameters :
            # ---------------------
            #' [Tune] Result: tweedie_variance_power=1.12; booster=gblinear;
            #' max_depth=4; min_child_weight=7.1; subsample=0.94;
            #' colsample_bytree=0.617; lambda=3.11; alpha=0.295; eta=0.314 :
            #' @mse.test.mean=3348276.3538605
            learner$par.vals <- list(
               objective = "reg:tweedie",
               eval_metric = "rmse",
               nrounds = 200,
               eta = 0.314,
               gamma = 1e-1,
               tweedie_variance_power=1.12,
               booster='gblinear',
               max_depth=4,
               min_child_weight=7.1,
               subsample=0.94,
               colsample_bytree=0.617,
               lambda=3.11,
               alpha=0.295
            )
         }
         
         # Train model
         xgb_model <- mlr::train(learner = learner, task = train_task)
         
         if (evaluate_perf){
            # Evaluation of performance for the occurrence detection.
            claim_predictions <- predict(xgb_model, valid_task)
            # RMSE
            rmse_tweedie <- mlr::performance(claim_predictions, mlr::rmse)
            print(rmse_tweedie)
            
            # Calculate measures for adequacy evaluation
            truth <- claim_predictions$data$truth
            mu <- claim_predictions$data$response
            xi <- xgb_model$learner.model$params$tweedie_variance_power
            phi <- optimize(function(phi) {
               -sum(log(1 + dtweedie(
                  y = truth,
                  xi = xi,
                  mu = mu,
                  phi = phi
               )))
            }, interval = c(0.1, 1000))$minimum
            U <- ecdf(truth)(truth)
            Fx <- function(x) ptweedie(x, xi=xi, mu=mu, phi = phi)
            
            # Kolmogorov-Smirnov test
            ks.test(U, Fx(truth))
            
            # Anderson-Darling test
            goftest::ad.test(truth, null=Fx, estimated=F, nullname = 'Tweedie')
            #' According to both the K-S and A-D test, the model doesn't fit 
            #' well to the dataset. 
            return(list("model"=xgb_model, "rmse"=rmse_tweedie))
         }
         return(list("model"=xgb_model))
   }
   
   
   severity_model <- train_xgb_tweedie(df_train,
                                       df_valid,
                                       gridsearch = GRIDSEARCH,
                                       evaluate_perf = EVALUATE_MODEL)$model
   occ_model <- train_xgb_occurrence(df_train, 
                                     df_valid, 
                                     gridsearch = GRIDSEARCH,
                                     evaluate_perf = EVALUATE_MODEL)$model

   return(list(occurence = occ_model,
               cost = severity_model))
}

In [15]:
model = fit_model(Xdata, ydata)

[21:21:37] WARNING: amalgamation/../src/learner.cc:541: 
Parameters: { colsample_bytree, gamma, max_depth, min_child_weight, subsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[1]	train-rmse:6363.581055 
[2]	train-rmse:3126.052734 
[3]	train-rmse:2488.643066 
[4]	train-rmse:2335.894531 
[5]	train-rmse:2257.385498 
[6]	train-rmse:2203.866211 
[7]	train-rmse:2164.817139 
[8]	train-rmse:2135.197021 
[9]	train-rmse:2112.111816 
[10]	train-rmse:2093.759766 
[11]	train-rmse:2079.039795 
[12]	train-rmse:2067.082031 
[13]	train-rmse:2057.322266 
[14]	train-rmse:2049.261963 
[15]	train-rmse:2042.603516 
[16]	train-rmse:2037.070312 
[17]	train-rmse:2032.441772 
[18]	train-rmse:2028.563721 
[19]	train-rmse:2025.290771 
[20]	train-rmse:2022.516479 
[21]	train-rmse:2020.157715 
[22]

## Saving your model

You can save your model to a file here, so you don't need to retrain it every time.

In [16]:
save_model <- function(model){
  # Saves this trained model to a file.

  # This is used to save the model after training, so that it can be used for prediction later.

  # Do not touch this unless necessary (if you need specific features). If you do, do not
  #  forget to update the load_model method to be compatible.
	
  # The default is to save it in 'trained_model.RData', but change this if you 
  # are using a pacakge that requires a different saving format.
  # For h2o models see this discussion: https://discourse.aicrowd.com/t/any-tips-for-successfully-submitting-an-h2o-model/4194/

  save(model, file='trained_model.RData')
}


In [17]:
save_model(model)

If you need to load it from file, you can use this code:

In [18]:
load_model <- function(){ 
 # Load a saved trained model from the file `trained_model.RData`.
 # This is called by the server to evaluate your submission on hidden data.


 # Only modify this *if* you modified save_model.

  load('trained_model.RData')
  return(model)
}

In [19]:
model = load_model()

# Predicting the claims 💵

The second function, `predict_expected_claim`, takes your trained model and a dataframe of contracts, and outputs a prediction for the (expected) claim incurred by each contract. This expected claim can be seen as the probability of an accident multiplied by the cost of that accident.

This is the function used to compute the _RMSE_ leaderboard, where the model best able to predict claims wins.

In [20]:
predict_expected_claim <- function(model, x_raw){
   #' Model prediction function: predicts the average claim based on the pricing model.
   #'
   #' This functions estimates the expected claim made by a contract (typically, as the product
   #' of the probability of having a claim multiplied by the average cost of a claim if it occurs),
   #' for each contract in the dataset X_raw.
   #'   
   #' This is the function used in the RMSE leaderboard, and hence the output should be as close
   #' as possible to the expected cost of a contract.
   #'
   #' Parameters
   #' ----------
   #' @X_raw : Dataframe, with the columns described in the data dictionary.
   #' 	Each row is a different contract. This data has not been processed.
   #'
   #' Returns
   #' -------
   #' @avg_claims: a one-dimensional array of the same length as X_raw, with one
   #'     average claim per contract (in same order). These average claims must be POSITIVE (>0).
   
   x_clean = preprocess_X_data(x_raw) %>% select(-year)
   
   expected_occ <- predict(model$occurence, newdata = x_clean)$data$prob.TRUE
   expected_loss <- predict(model$cost, newdata = x_clean)$data$response
   
   expected_claims = expected_occ * expected_loss
   return(round(expected_claims, 2))  
}

In [21]:
claims <- predict_expected_claim(model, Xdata)

# Pricing contracts 💰

The third and final function, `predict_premium`, takes your trained model and a dataframe of contracts, and outputs a _price_ for each of these contracts. **You are free to set this prices however you want!** These prices will then be used in competition with other models: contracts will choose the model offering the lowest price, and this model will have to pay the cost if an accident occurs.

This is the function used to compute the _profit_ leaderboard: your model will participate in many markets of size 10, populated by other participants' model, and we compute the average profit of your model over all the markets it participated in.

In [22]:
predict_premium <- function(model, x_raw){
  # Model prediction function: predicts premiums based on the pricing model.

  # This function outputs the prices that will be offered to the contracts in X_raw.
  # premium will typically depend on the average claim predicted in 
  # predict_expected_claim, and will add some pricing strategy on top.

  # This is the function used in the average profit leaderboard. Prices output here will
  # be used in competition with other models, so feel free to use a pricing strategy.

  # Parameters
  # ----------
  # X_raw : Dataframe, with the columns described in the data dictionary.
  # 	Each row is a different contract. This data has not been processed.

  # Returns
  # -------
  # prices: a one-dimensional array of the same length as X_raw, with one
  #     price per contract (in same order). These prices must be POSITIVE (>0).


  # YOUR CODE HERE ------------------------------------------------------

  # x_clean = preprocess_X_data(x_raw)  # preprocess your data before fitting

  return(predict_expected_claim(model, x_raw) * 1.1)
}


In [23]:
prices <- predict_premium(model, Xdata)
as.matrix(head(prices))

149.501
125.587
85.415
121.066
88.462
86.427


#### Profit on training data

In order for your model to be considered in the profit competition, it needs to make nonnegative profit over its training set. You can check that your model satisfies this condition below:

In [24]:
print(paste('Income:', sum(prices)))
print(paste('Losses:', sum(ydata)))

if (sum(prices) < sum(ydata)) {
    print('Your model loses money on the training data! It does not satisfy market rule 1: Non-negative training profit.')
    print('This model will be disqualified from the weekly profit leaderboard, but can be submitted for educational purposes to the RMSE leaderboard.')
} else {
    print('Your model passes the non-negative training profit test!')
}

[1] "Income: 45016367.242"
[1] "Losses: 26057988.08"
[1] "Your model passes the non-negative training profit test!"


# Ready? Submit to AIcrowd 🚀

If you are satisfied with your code, run the code below to send your code to the AICrowd servers for evaluation! This requires the variable `trained_model` to be defined by your previous code.

**Make sure you have included all packages needed to run your code in the [_"Packages"_](#packages) section.**

**NOTE**: If you submit the baseline RMSE model without any change whatsoever, your model will not be entered into the market. 

In [25]:
aicrowd_submit(AICROWD_API_KEY)

🚀 Preparing to submit...
⚙️ Collecting the submission code...
💾 Preparing the submission zip file...
Verifying API Key...
API Key valid
Saved API Key successfully!
                                          ╭─────────────────────────╮                                           
                                          │ Successfully submitted! │                                           
                                          ╰─────────────────────────╯                                           
                                                Important links                                                 
┌──────────────────┬───────────────────────────────────────────────────────────────────────────────────────────┐
│  This submission │ https://www.aicrowd.com/challenges/insurance-pricing-game/submissions/114363              │
│                  │                                                                                           │
│  All submissions │ https://www.aicrowd.com/